# Importing packages

## Setup

In [110]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import open3d
import pptk
from logging import raiseExceptions
import seaborn as sns


import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, datasets, metrics

## Data loader

In [ ]:
# def toSpherical(df):

#     theta = np.arctan2(np.sqrt(df['-X']**2 + df['Y']**2), df['Z'])

#     phi = np.arctan2(df['Y'],  df['-X'])

#     r = np.sqrt(df['-X']**2 + df['Y']**2 + df['Z']**2)

#     return (r, theta, phi)

# def toCartesian(df):
#     x = self.r*np.cos(self.phi)*sin(self.theta)
#     y = self.r*np.sin(self.phi)*sin(self.theta)
#     z = self.r*np.cos(self.theta)
#     return Pt(x,y,z)

In [155]:
class DataLoader():
    
    def __init__(self, modality = 'Radar', data_type='vector', normalize=True , dataset_directory='/home/artin/Documents/10dB_CR_10deg_in_2021-10-15-14-15-49', train_valid_test_ratio=[0.6, 0.2, 0.2]):

        self.dataset_directory = dataset_directory
        self.modality = modality
        self.data_type = data_type
        self.normalize = normalize
        self.train_valid_test_ratio = train_valid_test_ratio
        self.data = None
        
        if modality == 'Radar': 
            self.columns = ['Frame ID', 'Pocket ID' , 'Y', '-X', 'Z', 'Range', 'Azimuth Angle', 'Elevation Angle', 'RCS' , 'Velocity' , 'Range Index', 'Azimuth Index', 'Elevation Index', 'Velocity Index', 'Amplitude Index', 'Timestamp', 'Temperature', '?']
            self.rename_columns_dict = {'Range Index':'Range', 'Azimuth Index':'Azimuth', 'Elevation Index':'Elevation', 'Amplitude Index':'Amplitude', 'RCS':'RCS', 'Velocity Index':'Velocity'}
            
        elif modality == 'Lidar':
            self.columns = ['X', 'Y', 'Z', '?a' , '?b' , '?c']
            self.rename_columns_dict =  {'X':'X', 'Y':'Y', 'Z':'Z', '?a':'?a', '?b':'?b' , '?c':'?c'}
            
    def get_dataframe(self, dir: str):
        
        def _read_data(dir):
            self.dataframe_original = pd.read_csv(dir)        
            self.dataframe_original.columns = self.columns
            self.dataframe = self.dataframe_original.copy()
            
        def _filter_columns(columns: list):
            self.dataframe = self.dataframe[columns]
        
        def _rename_columns(columns: dict):
            self.dataframe.rename(columns=columns, inplace=True)
        
        _read_data(dir=dir)
        _filter_columns( columns=self.rename_columns_dict.keys() )
        _rename_columns( columns=self.rename_columns_dict )
            
    def get_data(self, filename = '1_.txt'):
        ''' data_type can be 'vector' or 'matrix' '''
        
        dir = f'{self.dataset_directory}/{self.modality}/{filename}'
        
        self.get_dataframe(dir=dir)
        data_raw = self.postprocess_dataframe(modality=self.modality, normalize=self.normalize)
        
        self.data = self.Data(data_raw=data_raw, train_valid_test_ratio=self.train_valid_test_ratio)

        return self.data
    
    def postprocess_dataframe(self, modality='Radar', normalize=True):
            
        def conversion_to_3D( matrix_dimentions={'Range':600, 'Azimuth':180, 'Elevation':20}, matrix_value='Amplitude'):
            
            ''' Range is assumed to be in the range [0, 600]
                Elevation Angle is in the range [-10, 10]
                Azimuth angle is in the range [0, 180]
                matrix_value can either be an integer/float or a string that corresponds to a column name. It's value will be used to fill the matrix. '''
            
            
            # Creating an empty array to store the data        
            self.data_3d = np.zeros( list(matrix_dimentions.values()) )
            
            # Shifting the Elevation values to the range (0, 20)
            self.dataframe.Elevation += 10
            
                    
            x,y,z = matrix_dimentions.keys()
            
            for _, row in self.dataframe.iterrows():
                self.data_3d [ int(row[x]) ] [ int(row[y]) ] [ int(row[z]) ] = row[matrix_value] if isinstance(matrix_value,str) else matrix_value
                
                        
            return self.data_3d
        
        def normalizing_to_0_1( normalization_values = {'Range':1000, 'Azimuth':180, 'Elevation':20, 'RCS':100, 'Velocity':200, 'Amplitude':100000} ):

            
            for key, value in normalization_values.items():
                self.dataframe[key] /= value
            
            self.dataframe.Elevation += 0.5
            self.dataframe.Velocity  += 0.5
        
            return self.dataframe
        

        if modality == 'Radar':
            
            if self.data_type == 'vector':  return normalizing_to_0_1() if self.normalize else self.dataframe
            
            elif self.data_type == 'matrix': return conversion_to_3D( matrix_dimentions={'Range':600, 'Azimuth':180, 'Elevation':20}, matrix_value=1)
            
            else: raise ValueError('data_type can be either "vector" or "matrix"')
            
        elif modality == 'Lidar':
            return self.dataframe
            
    class Data:
        def __init__(self, data_raw=None , train_valid_test_ratio: list=[3,1,1]):
            self.full  = data_raw
            self.train = None
            self.valid = None  
            self.test  = None      
            
            self._separate_train_valid_test_only_for_dataframe(train_valid_test_ratio=train_valid_test_ratio)          
                
        def _separate_train_valid_test_only_for_dataframe(self, train_valid_test_ratio=None):
            
            if train_valid_test_ratio is None:
                return 
            
            data = self.full.copy()
            
            frac = {}
            for ix, mode in enumerate(['train' , 'valid' , 'test']):
                frac[mode] = train_valid_test_ratio[ix]/sum(train_valid_test_ratio)
                
                
            self.train = data.sample(frac=frac['train'], random_state=42)    
            data.drop(self.train.index)

            self.valid = data.sample(frac=frac['valid'], random_state=42)
            data.drop(self.valid.index)
            
            self.test  = data.copy()
            
        @property
        def shape(self):
            return self.full.shape
            
    @staticmethod
    def appendSpherical_np(xyz):
        
        ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
        
        xy = xyz[:,0]**2 + xyz[:,1]**2
        
        ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
        
        # for elevation angle defined from Z-axis down
        # ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) 
        
        # for elevation angle defined from XY-plane up
        ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) 
        
        ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
        
        return ptsnew

    @classmethod
    def visualize(cls, points=None, method='open3d', run_demo=False, modality='Lidar', filename='1_.txt'):
        ''' 
        Example: Run from point clouds: 
            >> data = DataLoader(modality='Lidar').get_data(filename='1_.txt')
            >> DataLoader().visualize( points=data.train[data.train.columns[:3]].to_numpy()  , method='open3d' )
            
        Example: Run from demo sample:
            >> DataLoader().visualize(run_example=True, modality='Lidar', filename='2_.txt')
        '''
        
        if run_demo and points is None:
            data   = cls(modality=modality).get_data(filename=filename)
            points = data.train[data.train.columns[:3]].to_numpy()
 

        if method == 'open3d':
            
            pcd = open3d.geometry.PointCloud()
            pcd.points = open3d.utility.Vector3dVector(points)
            # pcd.colors = open3d.utility.Vector3dVector(data[data.columns[3]].to_numpy())
            open3d.visualization.draw_geometries([pcd])
            

        elif method == 'pptk':
            
            # points = pptk.points(points)
            v = pptk.viewer(points)
            # v.attribute('color', data[data.columns[3]].to_numpy())
            
        else:
            raiseExceptions('method should be either "open3d" or "pptk"')

## <span >

In [152]:
# Example 1:
# data = DataLoader(modality='Lidar').get_data(filename='1_.txt')
# points=data.train[data.train.columns[:3]].to_numpy()
# DataLoader().visualize(points=points , method='open3d')

# Example 2:
DataLoader().visualize(run_demo=True, modality='Radar', filename='2_.txt')


In [157]:
data = DataLoader(modality='Lidar').get_data(filename='1_.txt')

data.full

,X,Y,Z,?a,?b,?c
0,72.6694,-70.6876,3.8644,18,1.504763e+09,5
1,74.1860,-62.3421,2.5476,18,1.504763e+09,7
2,72.6927,-70.7103,2.0676,13,1.504763e+09,8
3,74.0649,-62.2404,0.8308,16,1.504763e+09,10
4,72.5756,-70.5965,0.2739,12,1.504763e+09,11
...,...,...,...,...,...,...
13693,3.5100,-3.3645,-1.0203,7,1.504763e+09,35
13694,3.1845,-3.0525,-1.0037,8,1.504763e+09,36
13695,2.9560,-2.8335,-1.0039,7,1.504763e+09,37
13696,2.1002,-2.0132,-0.9866,8,1.504763e+09,38


## Build the autoencoder

We are going to use the Functional API to build our convolutional autoencoder.

In [ ]:
class Optimization():
    
    def __init__(self):   
        self.model = None
                       
    def fit(self, epochs=5, batch_size=32, train=None, valid=None):
        
        self.model = self.architecture(input_shape=(train.shape[1],1))
        
        self.model.fit(x=train, y=train, epochs=epochs, batch_size=batch_size, validation_data=(valid, valid))
        
    def architecture(self, input_shape: tuple):
    
        input = layers.Input(shape=input_shape)

        # Encoder
        x = layers.Conv1D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu')(input)
        x = layers.Conv1D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu')(x)
        x = layers.Conv1D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu')(x)

        # Decoder
        x = layers.Conv1DTranspose(filters=256, strides=2, kernel_size=3, padding='same', activation='relu')(x)
        x = layers.Conv1DTranspose(filters=256, strides=1, kernel_size=3, padding='same', activation='relu')(x)
        x = layers.Conv1DTranspose(filters=1,   strides=1, kernel_size=3, padding='same', activation='sigmoid')(x)


        # Autoencoder
        model = models.Model(input, x)
        model.compile(optimizer="adam", loss="binary_crossentropy")
        
        return model
    
    def predict(self, test):
        
        predictions = self.model.predict(test)
        
        if isinstance(test, pd.DataFrame):
            return pd.DataFrame(predictions[:,:,0], columns=test.columns, index=test.index)
        return predictions

## <span style="color:orange; font-size:0.8em"> Vector Input </span>

In [117]:
class Vector_Input(DataLoader, Optimization):
    
    def __init__(self, dataset_directory='/home/artin/Documents/10dB_CR_10deg_in_2021-10-15-14-15-49' , modality='Radar', normalize=True, filename='1_.txt', train_valid_test_ratio=[3,1,1]):

        # Loading the data                
        DataLoader.__init__( self, data_type='vector' , dataset_directory=dataset_directory, modality=modality, normalize=normalize, train_valid_test_ratio=train_valid_test_ratio)
        self.get_data(filename=filename)     

        # Getting the architecture of the model
        Optimization.__init__(self)
        
        # Training the model
        self.fit(epochs=5, batch_size=32, train=self.data.train, valid=self.data.valid)
        
        # Testing the model
        self.prediction = self.predict(test=self.data.test)
           
    
    def view(self):             

        test = self.data.test.reset_index()
        predictions = self.prediction.reset_index()

        sns.set()
        for y in test.columns[1:]:

            plt.figure(figsize=(12, 6))
            sns.scatterplot(x='index', y=y, data=test, label='data.test')
            sns.scatterplot(x='index', y=y, data=predictions, label='predictions')
            plt.legend()
            plt.xlabel('index')
            plt.show()
            

vector_input = Vector_Input(filename='1_.txt', modality='Radar', normalize=True)

vector_input.prediction

Epoch 1/5
20/20 [==============================] - 3s 116ms/step - loss: 0.5993 - val_loss: 0.4750
Epoch 2/5
20/20 [==============================] - 2s 104ms/step - loss: 0.4724 - val_loss: 0.4587
Epoch 3/5
20/20 [==============================] - 2s 99ms/step - loss: 0.4562 - val_loss: 0.4499
Epoch 4/5
20/20 [==============================] - 2s 106ms/step - loss: 0.4536 - val_loss: 0.4479
Epoch 5/5
20/20 [==============================] - 2s 99ms/step - loss: 0.4471 - val_loss: 0.4474


,Range,Azimuth,Elevation,Amplitude,RCS,Velocity
0,0.015083,0.181331,0.758916,0.145769,0.844034,0.505733
1,0.011155,0.311069,0.078670,0.011699,0.490392,0.791993
2,0.011366,0.318913,0.078736,0.011431,0.477683,0.792275
3,0.013570,0.327488,0.414730,0.008916,0.440100,0.784166
4,0.013519,0.329844,0.414361,0.008941,0.441903,0.784158
...,...,...,...,...,...,...
1061,0.436967,0.258296,0.626389,0.026462,0.649983,0.527063
1062,0.457369,0.224252,0.442214,0.026891,0.643337,0.537056
1063,0.398510,0.257622,0.784968,0.021538,0.627484,0.515747
1064,0.395959,0.092662,0.670510,0.017200,0.612228,0.798983


In [116]:
# vector_input.view()

## <span style="color:orange; font-size:0.8em"> Matrix Input </span>

In [118]:
class Matrix_Input(DataLoader, Optimization):
    
    def __init__(self, dataset_directory='/home/artin/Documents/10dB_CR_10deg_in_2021-10-15-14-15-49' , modality='Radar', train_valid_test_ratio=None, normalize=True, filename='1_.txt'):

        # Loading the data                
        DataLoader.__init__(self, data_type='matrix' , modality=modality, dataset_directory=dataset_directory, normalize=True, train_valid_test_ratio=train_valid_test_ratio)
        self.get_data(filename=filename)  
        
        # Getting the architecture of the model
        Optimization.__init__(self)
        
        # Training the model
        # self.fit(epochs=5, batch_size=32, train=self.data.train, valid=self.data.valid)
        
        # Testing the model
        # self.predictions = self.predict(test=self.data.test)
        
        
matrix_input = Matrix_Input(dataset_directory='/home/artin/Documents/10dB_CR_10deg_in_2021-10-15-14-15-49' , modality='Radar')

matrix_input.data.full

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.